# LRT Request Analysis

## Functionalities
- Analysis of RPCs and queries of LRT requests.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
    ...
    loadgen.tar.gz
  ...
  [node-n]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
    ...
    loadgen.tar.gz
```
`*_service*.tar.gz` and `apigateway*.tar.gz` tarballs contain RPC log files named `calls.log` and database query log files named `queries.log`. A tarball `loadgen.tar.gz` contains a request log file named `loadgen.log`.

## Notebook Configuration

In [1]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_2021-11-23-16-32-52"

## Notebook Setup

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath(os.path.join("..")))
from parsers.loadgen_parser import LoadgenParser
from parsers.query_parser import QueryParser
from parsers.rpc_parser import RPCParser
from utils.utils import *

## Request Log Parsing

In [3]:
# Build data frame
requests = pd.concat(
    [pd.DataFrame.from_dict(LoadgenParser(logfile).parse()) for logfile in get_loadgen_logfiles(EXPERIMENT_DIRNAME)],
    ignore_index=True)

## RPC Log Parsing

In [4]:
# Build data frame
rpc = pd.concat(
        [pd.DataFrame.from_dict(RPCParser(logfile).parse()) for logfile in get_rpc_logfiles(EXPERIMENT_DIRNAME)], 
        ignore_index=True)

## Query Log Parsing

In [5]:
# Build data frame
query = pd.concat(
        [pd.DataFrame.from_dict(QueryParser(logfile).parse()) for logfile in get_query_logfiles(EXPERIMENT_DIRNAME)],
        ignore_index=True)

## Analysis of LRT Requests

In [11]:
# Analysis of requests with latency over P99.9
start_time = get_experiment_start_time(EXPERIMENT_DIRNAME)
lrt_requests = requests[(requests["latency"] > requests[requests["status_code"] == 200]["latency"].quantile(0.999))]
for lrt_request in lrt_requests.to_dict("records"):
    if lrt_request["type"] == "retrieve_recent_posts":
        print(lrt_request)
        print("Request ID: %s" % lrt_request["request_id"])
        print("  Type: %s" % lrt_request["type"])
        print("  Experiment Timestamp: %s" % (lrt_request["timestamp"]-start_time).total_seconds())
        print("  RPCs:")
        for lrt_request_rpc in rpc[(rpc["request_id"] == lrt_request["request_id"])].to_dict("records"):
            print("    %s - %s" % (lrt_request_rpc["function"], lrt_request_rpc["latency"]))
        print("  Queries:")
        for lrt_request_query in query[(query["request_id"] == lrt_request["request_id"])].to_dict("records"):
            print("    %s - %s" % (lrt_request_query["dbname"] + ":" + lrt_request_query["type"],
                    lrt_request_query["latency"]))
for lrt_request in lrt_requests.to_dict("records"):
    if lrt_request["type"] == "retrieve_recent_posts":
        print("%s <-> %s" % ((lrt_request["timestamp"]-start_time).total_seconds(), lrt_request["latency"]))

{'timestamp': Timestamp('2021-11-23 16:56:30.417000'), 'method': 'GET', 'url': 'http://pc553.emulab.net:8888/post', 'request_id': 'uQQDZxLc', 'status_code': 200, 'latency': 0.107, 'status': 'successful', 'type': 'retrieve_recent_posts', 'rw': 'read'}
Request ID: uQQDZxLc
  Type: retrieve_recent_posts
  Experiment Timestamp: 328.879
  RPCs:
    account:authenticate_user - 39.865618
    post:list_posts - 50.091346
    account:retrieve_standard_account - 8.80245
    account:retrieve_standard_account - 8.868149
    account:retrieve_standard_account - 10.521278
    account:retrieve_standard_account - 10.587565
    account:retrieve_standard_account - 11.640714
    account:retrieve_standard_account - 13.499713999999999
    account:retrieve_standard_account - 16.752578
    like:count_likes_of_post - 16.511267999999998
    account:retrieve_standard_account - 18.981573
    like:count_likes_of_post - 19.699023
    like:count_likes_of_post - 24.562810000000002
    like:count_likes_of_post - 25.768